# Merging, cleaning, and lagging data

#### Inputs
Use defaults

In [1]:
def try_(input_, default = []):
    if not input_:
        return default
    return input_

# Start date
start = input("Start date for data slicing, default = 2000-01-01: ")
start = try_(start, "2000-01-01")

# End date
end = input("End date for data slicing, default = 2020-12-31: ")
end = try_(end, "2020-12-31")

# Number of lags
n_lags = input("Number of lags for daily data, default = 5: ")
if n_lags == "":
    pass
else:
    n_lags = int(n_lags)
n_lags = try_(n_lags, 5)

# Variables to drop
drop_variables = input("Variables to drop, default =  chinaHK_CPI and STYRINGSRENTEN: ").split()
drop_variables = try_(drop_variables, "chinaHK_CPI STYRINGSRENTEN")
    
# Frequency
frequency = input("Data frequency. 0 = daily, 1 = by publication, default = 1: ")
if frequency == "":
    pass
else:
    frequency = int(frequency)
frequency = try_(frequency, 1)

Start date for data slicing, default = 2000-01-01:  
End date for data slicing, default = 2020-12-31:  
Number of lags for daily data, default = 5:  
Variables to drop, default =  chinaHK_CPI and STYRINGSRENTEN:  
Data frequency. 0 = daily, 1 = by publication, default = 1:  


#### Import packages and data

In [2]:
import pandas as pd
import os

In [3]:
# === Data ===
# Read the whole excel file top_data
os.chdir("/Users/benjaminlian/Documents/School/5th_year/Spring_2021/Master_Thesis")
xls = pd.ExcelFile("Data/top_data_3.xlsx")

# Construct the seperate dataframes and get all column names
dic = {}
col_names = []
sheets = xls.sheet_names
sheets.remove("Innhold")

for i in range(len(sheets)):
    dic[sheets[i]] = pd.read_excel(xls, sheets[i], index_col = 0)
    col_names += list(dic[sheets[i]].columns)

# Construct empty dataframe with full timeline
import datetime as dt
import numpy as np

datelist = pd.date_range(start = dt.datetime(1990, 1, 1), end = dt.datetime(2021, 1, 15)).tolist()
datelist = pd.DataFrame(np.zeros(len(datelist)), index = datelist)

# Create dataframe of Forklaringsvariabler_5 formatting and lags
lags = dic["Forklaringsvariabler"].loc[:, ["Variablenavn", "Publiseringslag"]].reset_index(drop = True)
lags.dropna(inplace = True)
freq = dic["Forklaringsvariabler"].loc[:, ["Variablenavn", "Frekvens"]].reset_index(drop = True)
freq.dropna(inplace = True)

# Create series for NO_folio_publication_dates
pub_dates = dic["Sheet22"].sort_index() / 100

# Remove sheets from the top_data dictionary
del dic["Forklaringsvariabler"]
sheets.remove("Forklaringsvariabler")
del dic["Sheet22"]
sheets.remove("Sheet22")

#### Data Wrangling

In [4]:
def period_return(vector, frequency):
    if frequency == "d":
        pr_dict = dict()
        pr_dict[vector.name + "_d_2_d"]  = vector.pct_change(1)
        pr_dict[vector.name + "_d_2_w"]  = vector.pct_change(5)
        pr_dict[vector.name + "_d_2_2w"] = vector.pct_change(10)
        pr_dict[vector.name + "_d_2_m"]  = vector.pct_change(30)
        pr_dict[vector.name + "_d_2_2m"] = vector.pct_change(60)
        pr_dict[vector.name + "_d_2_k"]  = vector.pct_change(90)
        pr_dict[vector.name + "_d_2_2k"] = vector.pct_change(183)
        pr_dict[vector.name + "_d_2_y"]  = vector.pct_change(365)
        
        for n in range(n_lags):
            pr_dict[vector.name + "_d_2_d_lag{}".format(n + 1)] = pr_dict[vector.name + "_d_2_d"].shift(n + 1)
    
    elif frequency == "u":
        pr_dict = dict()
        pr_dict[vector.name + "_w_2_w"]  = vector.pct_change(1)
        pr_dict[vector.name + "_w_2_2w"] = vector.pct_change(2)
        pr_dict[vector.name + "_w_2_m"]  = vector.pct_change(4)
        pr_dict[vector.name + "_w_2_2m"] = vector.pct_change(9)
        pr_dict[vector.name + "_w_2_k"]  = vector.pct_change(13)
        pr_dict[vector.name + "_w_2_2k"] = vector.pct_change(26)
        pr_dict[vector.name + "_w_2_y"]  = vector.pct_change(52)
    
    elif frequency == "m":
        pr_dict = dict()
        pr_dict[vector.name + "_m_2_m"]  = vector.pct_change(1)
        pr_dict[vector.name + "_m_2_2m"] = vector.pct_change(2)
        pr_dict[vector.name + "_m_2_k"]  = vector.pct_change(3)
        pr_dict[vector.name + "_m_2_2k"] = vector.pct_change(6)
        pr_dict[vector.name + "_m_2_y"]  = vector.pct_change(12)
    
    elif frequency == "k":
        pr_dict = dict()
        pr_dict[vector.name + "_k_2_k"]  = vector.pct_change(1)
        pr_dict[vector.name + "_q_2_2k"] = vector.pct_change(2)
        pr_dict[vector.name + "_q_2_y"]  = vector.pct_change(4)
    
    elif frequency == "å":
        pr_dict = dict()
        pr_dict[vector.name + "_å_2_å"]  = vector.pct_change(1)

    return pr_dict

In [5]:
# === Formatting ===
# Periodic returns
dic_2 = dict()
for sheet in dic:
    dic_vars = dict()
    for variable in dic[sheet]:
        if variable in drop_variables:
            pass
        else:
            series = dic[sheet].loc[:, variable]
            frequency_ = freq.loc[freq.loc[:, "Variablenavn"] == variable, :].Frekvens.item()
            if series.index[0] > series.index[1]:
                series = series.sort_index()
            dict_vectors = period_return(series, frequency_)
            for key in dict_vectors:
                if np.isinf(dict_vectors[key]).any():
                    dict_vectors[key].replace([np.inf, -np.inf], 0, inplace = True)
            dic_vars[variable] = pd.DataFrame(dict_vectors)
    
    df_ = pd.DataFrame()
    for key in dic_vars:
        _ = dic_vars[key]
        df_ = pd.concat([df_, _], axis = 1)
    dic_2[sheet] = df_

# Concatenate sheets with full timeline dataframe
df_full = datelist

for sheet in dic_2:
    df_full = pd.concat([dic_2[sheet], df_full], axis = 1, join = "outer")

# Drop specified variables
df_full.drop(columns = [0], inplace = True)
    
# === Lag variables (publication lags), drop specified variables and fill missing values ===
df_copy = df_full.copy()
for idx, lag in enumerate(lags.Publiseringslag):
    if lags.Variablenavn.iloc[idx] in df_copy.columns:
        variables = [column for column in df_copy.columns if lags.iloc[idx, 0] in column]
        if lags.iloc[idx, 0] in drop_variables:
            df_copy[:, variables] = []
        df_copy.loc[:, variables] = df_copy.loc[:, variables].shift(periods = lag, freq = "D")

# Fill missing
df_copy.dropna(how = "all", inplace = True)
df_filled = df_copy.ffill()

# === Cut the dataframe ===
# Define function that slices and drops variables given a period
def wrangler(df, start, end):
    print("Variables deleted:")
    df_ = df.copy()
    sliced_df_dict = {}
    for i in df_.columns:
        if df_.loc[start:end, i].isna().any():
            print(i)
        else:
            sliced_df_dict[i] = df_.loc[start:end, i]
    
    sliced_df = pd.concat(sliced_df_dict, axis = 1)
    
    return sliced_df

# Slice
# Note: Vil ha med NO_folio_publication_dates med nan i mellom publiseringer
sliced_data = wrangler(df_filled, start, end)

# === Merge with publication dates ===
if frequency == 1:
    sliced_data = pd.merge(pub_dates, sliced_data, left_index = True, right_index = True)
else:
    temp = pd.DataFrame(np.full([len(df_filled), 1], np.nan), index = df_filled.index)
    temp = temp.join(pub_dates, how = "outer")
    temp.NO_folio_publication_date.ffill(inplace = True)
    temp.rename(columns = {"NO_folio_publication_date":"NO_folio_rate"}, inplace = True)
    temp.drop(columns = [0], inplace = True)
    sliced_data = pd.concat([sliced_data, pub_dates[start:end], temp[start:end]], axis = 1)

Variables deleted:
ECB_QE_å_2_å
oil_inv_k_2_k
oil_inv_q_2_2k
oil_inv_q_2_y
EUR_NOK_d_2_y
NO_EL_price_k_2_k
NO_EL_price_q_2_2k
NO_EL_price_q_2_y
NO_KPI_JA_m_2_m
NO_KPI_JA_m_2_2m
NO_KPI_JA_m_2_k
NO_KPI_JA_m_2_2k
NO_KPI_JA_m_2_y
NO_KPI_JAE_m_2_m
NO_KPI_JAE_m_2_2m
NO_KPI_JAE_m_2_k
NO_KPI_JAE_m_2_2k
NO_KPI_JAE_m_2_y
sweden_folio_rate_m_2_m
sweden_folio_rate_m_2_2m
sweden_folio_rate_m_2_k
sweden_folio_rate_m_2_2k
sweden_folio_rate_m_2_y
interest_rates_on_loans_m_2_m
interest_rates_on_loans_m_2_2m
interest_rates_on_loans_m_2_k
interest_rates_on_loans_m_2_2k
interest_rates_on_loans_m_2_y
FED_QE_k_2_k
FED_QE_q_2_2k
FED_QE_q_2_y
fish_pool_NOK_w_2_w
fish_pool_NOK_w_2_2w
fish_pool_NOK_w_2_m
fish_pool_NOK_w_2_2m
fish_pool_NOK_w_2_k
fish_pool_NOK_w_2_2k
fish_pool_NOK_w_2_y


#### Data cleaning and slicing

In [6]:
# Output
sliced_data.to_excel("./Data/master_thesis_data.xlsx")